# `20180811a` - Checking Kretz probe - server

In [1]:
#!/usr/bin/python
import spidev
import RPi.GPIO as GPIO
import time
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import json
import time
from pyUn0 import *

pyexiv2 does not exist on RPi


In [2]:
x = us_spi()
x.JSON = {}
x.JSON["firmware_md5"]="fa6a7560ade6d6b1149b6e78e0de051f"
x.JSON["firmware_version"]="e_un0"
x.JSON["data"]=[]
x.JSON["registers"]={}
x.JSON["parameters"]={}
x.JSON["experiment"]={} = "20180811a"
x.JSON["experiment"]["description"]="Testing several wires"
x.JSON["experiment"]["probe"]="test aw"
x.JSON["experiment"]["target"] = "none"
x.JSON["experiment"]["position"] = "0"
x.JSON["V"]="48"

In [3]:
x.StartUp()
x.ConfigSPI()

# Setting acquition speed
f = 0x00
x.WriteFPGA(0xED,f) # Frequency of ADC acquisition / sEEADC_freq (3 = 16Msps, 1 = 32, 0 = 64, 2 = 21Msps)

x.WriteFPGA(0xEB,0x00) # Doing one line if 0, several if 1
x.WriteFPGA(0xEC,0x01) # Doing 1 lines
if x.JSON["registers"][235]: # means it's set to 1, ie that's multiples lines
    NLines = x.JSON["registers"][236]
else:
    NLines = 1

Fech = int(64/((1+f)))


pyUn0.py:102: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(CS_FLASH,GPIO.OUT)


Reset GPIO 23 - Low 1s
Reset GPIO 23 - High 0.2s
spi.cshigh is False
spi mode is 1
spi maxspeed is 2000000hz
Pulse width: 200  ns --  0x19
Pulses delay: 100  ns --  0xc
Poff: 2000  ns --  0x1 0x0
Delay between: 7000 ns --  0x3 0x80
Config FPGA done!


In [13]:
x.JSON["N"] = 0 # Experiment ID

# Timings
t1 = 200
t2 = 100
t3 = 2000
t4 = 3000-t1-t2-t3
t5 = 200000

LAcq = t5/1000 #ns to us 
Nacq = LAcq * Fech * NLines

# Setting up the DAC, from 50mV to 850mv
Curve = x.CreateDACCurve(200,800,True)[0]
x.setDACCurve(Curve)
# Setting pulses
x.setPulseTrain(t1,t2,t3,t4,t5)

Pulse width: 200  ns --  0x19
Pulses delay: 295  ns --  0x25
Poff: 2484  ns --  0x1 0x3d
Delay between: 3660 ns --  0x1 0xd4
Acquisition length:  3385.088  us --  0x67 0x4e


206609

In [20]:
x.JSON["N"] = x.JSON["N"] +1
# Trigger
x.WriteFPGA(0xEF,0x01) # Cleaning memory pointer
x.WriteFPGA(0xEA,0x01) # Software Trig : As to be clear by software

A = []
for i in range(2*Nacq+1):
    A.append ( x.spi.xfer([0x00] )[0] )
a = np.asarray(A).astype(np.int16)

x.JSON["data"] = A

x.JSON["experiment"]["description"]="Testing yellow coax"

with open(x.JSON["experiment"]["id"]+str(x.JSON["N"])+".json", 'w') as outfile:
    json.dump(x.JSON, outfile)
    
print x.JSON["experiment"]["id"]+"-"+str(x.JSON["N"])+".json:  DONE"

20180811a-7.json:  DONE
